# Use Quinton Lawton's QTrack to track AEWs in reanalysis data over BOWTIE time period

Data requirements: 700hPa wind (u and v component), 6 hourly timesteps, 1 degree by 1 degree resolution. 
BOWTIE time period is 6Z August 16, 2024 - 3Z September 24, 2024 (end of measurements)

In [4]:
import qtrack
from qtrack.curvvort import compute_curvvort
from qtrack.tracking import run_postprocessing, run_tracking
import xarray as xr

In [5]:
#Load data
#uvdata = xr.open_mfdataset('era5_700_wind_global_2010_10day.nc')
uvdata = xr.open_mfdataset('../../data/ERA5_6hrly_700hPa_winds_20240809.nc')
uvdata

<xarray.Dataset> Size: 283MB
Dimensions:         (valid_time: 244, pressure_level: 1, latitude: 241,
                     longitude: 601)
Coordinates:
    number          int64 8B ...
  * valid_time      (valid_time) datetime64[ns] 2kB 2024-08-01 ... 2024-09-30...
  * pressure_level  (pressure_level) float64 8B 700.0
  * latitude        (latitude) float64 2kB 50.0 49.75 49.5 ... -9.5 -9.75 -10.0
  * longitude       (longitude) float64 5kB -90.0 -89.75 -89.5 ... 59.75 60.0
    expver          (valid_time) <U4 4kB dask.array<chunksize=(244,), meta=np.ndarray>
Data variables:
    u               (valid_time, pressure_level, latitude, longitude) float32 141MB dask.array<chunksize=(122, 1, 121, 301), meta=np.ndarray>
    v               (valid_time, pressure_level, latitude, longitude) float32 141MB dask.array<chunksize=(122, 1, 121, 301), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-08T10:55 GRIB to CDM+CF via cfgrib-0.9.1...

In [6]:
#Coarsen data from 0.25 degrees to 1 degree
uv_coarse = uvdata.coarsen(longitude=4,latitude=4,boundary='pad').mean()
uv_coarse = uv_coarse.rename({'valid_time':'time','pressure_level':'level'})
uv_coarse.to_netcdf('prepped_data.nc')
uv_coarse

<xarray.Dataset> Size: 18MB
Dimensions:    (time: 244, level: 1, latitude: 61, longitude: 151)
Coordinates:
    number     int64 8B 0
  * time       (time) datetime64[ns] 2kB 2024-08-01 ... 2024-09-30T18:00:00
  * level      (level) float64 8B 700.0
  * latitude   (latitude) float64 488B 49.62 48.62 47.62 ... -8.375 -9.375 -10.0
  * longitude  (longitude) float64 1kB -89.62 -88.62 -87.62 ... 58.38 59.38 60.0
    expver     (time) <U4 4kB dask.array<chunksize=(244,), meta=np.ndarray>
Data variables:
    u          (time, level, latitude, longitude) float32 9MB dask.array<chunksize=(122, 1, 30, 75), meta=np.ndarray>
    v          (time, level, latitude, longitude) float32 9MB dask.array<chunksize=(122, 1, 30, 75), meta=np.ndarray>
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-11-08T10:55 GRIB to CDM+CF via cfgrib-0.9.1...

### Prep data 

In [7]:
prepped_data_save = "adjusted_data.nc"
qtrack.prep_data(data_in="prepped_data.nc", data_out=prepped_data_save, cut_lev_val=700)

Possibility of uncut level. Check names of dimensions.
We have a level file, slice!
Possibility of uncut level. Check names of dimensions.
We have a level file, slice!
Prepped data saved to: adjusted_data.nc


### Curvature vorticity calculation

In [8]:
data_file_in = "adjusted_data.nc"
curv_file_out = "curv_vort_era5_BOWTIE.nc"
compute_curvvort(data_file_in, curv_file_out, njobs_in=-1)

Starting Computation of Radial Averaged CV...
Timestep number: 1
Timestep number: 2
Timestep number: 0
Timestep number: 3
Timestep number: 4
Timestep number: 5
Timestep number: 6
Timestep number: 7
Timestep number: 8
Timestep number: 9
Timestep number: 10
Timestep number: 11
Timestep number: 12
Timestep number: 13
Timestep number: 14
Timestep number: 15
Timestep number: 16
Timestep number: 17
Timestep number: 18
Timestep number: 19
Timestep number: 20
Timestep number: 21
Timestep number: 22
Timestep number: 23
Timestep number: 24
Timestep number: 25
Timestep number: 26
Timestep number: 27
Timestep number: 28
Timestep number: 29
Timestep number: 30
Timestep number: 31
Timestep number: 32
Timestep number: 33
Timestep number: 34
Timestep number: 35
Timestep number: 36
Timestep number: 37
Timestep number: 38
Timestep number: 39
Timestep number: 40
Timestep number: 41
Timestep number: 42
Timestep number: 43
Timestep number: 44
Timestep number: 45
Timestep number: 46
Timestep number: 47
Time

### AEW Tracking step

In [9]:
AEW_raw_save_file = "AEW_tracks_raw.nc"
run_tracking(input_file=curv_file_out, save_file=AEW_raw_save_file)

1 out of 244
2 out of 244
3 out of 244
4 out of 244
5 out of 244
6 out of 244
7 out of 244
8 out of 244
9 out of 244
10 out of 244
11 out of 244
12 out of 244
13 out of 244
14 out of 244
15 out of 244
16 out of 244
17 out of 244
18 out of 244
19 out of 244
20 out of 244
21 out of 244
22 out of 244
23 out of 244
24 out of 244
25 out of 244
26 out of 244
27 out of 244
28 out of 244
29 out of 244
30 out of 244
31 out of 244
32 out of 244
33 out of 244
34 out of 244
35 out of 244
36 out of 244
37 out of 244
38 out of 244
39 out of 244
40 out of 244
41 out of 244
42 out of 244
43 out of 244
44 out of 244
45 out of 244
46 out of 244
47 out of 244
48 out of 244
49 out of 244
50 out of 244
51 out of 244
52 out of 244
53 out of 244
54 out of 244
55 out of 244
56 out of 244
57 out of 244
58 out of 244
59 out of 244
60 out of 244
61 out of 244
62 out of 244
63 out of 244
64 out of 244
65 out of 244
66 out of 244
67 out of 244
68 out of 244
69 out of 244
70 out of 244
71 out of 244
72 out of 244
7

### AEW Postprocessing step

In [10]:
AEW_final_nc_file = "AEW_tracks_BOWTIE_post_processed.nc"
run_postprocessing(input_file=AEW_raw_save_file, curv_data_file=curv_file_out, object_data_save=False, save_nc_file=AEW_final_nc_file,hov_over_africa_color=False)

## Done! A diagram should have been saved as "final_hovmoller.png"